# Road Following - Data Collection（道路跟踪-数据采集）

如果您已经浏览过避免碰撞示例，您应该熟悉以下三个步骤
1. 数据收集
2. 训练
3. 部署

在这个笔记本里，我们会做同样的事情！除了，您将学习一种不同的基本技术 **regression** ，而不是分类，我们将使用它来使JetBot能够沿着一条路（或任何路径或目标点）行驶。
1. 将JetBot放置在路径上的不同位置（偏离中心、不同角度等）
> 切记避撞的范例，数据变化是关键！
2. 显示来自机器人的实时摄像机馈送
3. 使用游戏板控制器，在图像上放置一个“绿点”，它对应于我们希望机器人行进的目标方向。
4. 存储这个绿点的X，Y值以及机器人摄像机的图像

然后，在训练笔记本中，我们将训练一个神经网络来预测标签的X，Y值。在现场演示中，我们将使用预测的x，y值来计算一个近似的转向值（它不是‘正好’一个角度，如这需要图像校准，但它与角度大致成正比，这样我们的控制器就能正常工作）。那么，对于本例，您如何确定目标的确切位置？这是一本我们认为可能有用的指南
1. 从摄像机看现场视频
2. 想象一下机器人应该遵循的道路（尽量接近它需要避开跑道等的距离）。
3. 将目标尽可能远地沿着这条路径放置，这样机器人就可以直奔目标，而不会“跑开”道路。
>例如，如果我们在一条非常直的路上，我们可以把它放在地平线上。如果我们在急转弯时，可能需要把它放在离小车更近的地方，这样它就不会跑出边界。

假设我们的深度学习模型按预期工作，这些标签指南应确保：
1. 机器人可以安全地直接向目标移动（不越界等）
2. 目标将沿着我们想象的道路不断前进
我们得到的，是一个“大棒上的胡萝卜”，沿着我们期望的轨迹移动。深度学习决定了胡萝卜的摆放位置，而JetBot则紧随其后：）

### 标记示例视频（需要翻墙才能看，如果看不到就跳过，不影响后面的操作）
执行代码块以查看如何标记图像的示例。此模型仅采集123个图像后就能工作：）

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

### 导入库
因此，让我们从导入“数据收集”所需的所有库开始。我们将主要使用OpenCV来可视化和保存带有标签的图像。库（如uuid、datetime）用于图像命名。

In [ ]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### 实时显示摄像头的影像
首先，让我们像在远程操作笔记本中那样初始化和显示相机。

我们使用来自JetBot的Camera类来启用CSI MIPI Camera。神经网络以224x224像素的图像作为输入，将把相机设置为该大小，以最小化数据集的文件大小（我们已经测试过它是否适用于此任务）。在某些情况下，最好以更大的图像大小收集数据，然后缩小到所需的大小。

In [ ]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

### 创建游戏板控制器

此步骤类似于“Teleoperation”任务，都是使用gamepad控制器来标记图像。

我们要做的第一件事是创建一个Controller小部件的实例，我们将使用它来标记带有 “x” 和 “y” 值的图像，如引言中所述。Controller小部件接受一个索引参数，该参数指定控制器的编号。这对于连接多个控制器或某些游戏板显示为多个控制器的情况非常有用。要确定正在使用的控制器的索引，

1. 访问http://html5gamepad.com。
2. 按你正在使用的游戏板上的按钮
3. 记住按钮按下时游戏板的索引
4. 接下来，我们将使用该索引创建并显示控制器。

In [ ]:
controller = widgets.Controller(index=0)

display(controller)

### 将Gamepad控制器连接到标签图像
现在，即使我们已经连接了我们的游戏板，我们还没有连接到标签图像控制器！我们将使用dlink函数将其连接到左右垂直轴。dlink函数不同于link函数，它允许我们在源和目标之间附加一个转换。

In [ ]:
widgets.jsdlink((controller.axes[2], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (y_slider, 'value'))

### 收集数据

下面的代码块将显示实时图像源以及我们保存的图像数。我们将目标X，Y值存储在：
1. 把绿点放在目标上
2. 按DPAD上的“向下”键保存
这将在“dataset-xy”文件夹中存储一个名为

``xy值><x值><y值><uuid>.jpg``

当我们训练时，我们加载图像并解析文件名中的x，y值

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[13].observe(save_snapshot, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

### Next
一旦你收集了足够的数据，我们需要将这些数据复制到我们的GPU桌面或云机器上进行培训。首先，我们可以调用以下终端命令将数据集文件夹压缩为一个zip文件。

> 如果你要在 Jetbot 上训练模型，就可以跳过这一步！

这个！prefix表示我们希望将单元格作为shell（或terminal）命令运行。
下面zip命令中的-r标志表示递归以便包含所有嵌套文件，-q标志表示安静以便zip命令不打印任何输出

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

在Jupyter实验室文件浏览器中，您应该会看到一个名为road_following_u<Date&Time>.zip的文件。您应该使用Jupyter实验室文件浏览器通过右键单击并选择下载来下载zip文件。